# Top

In [1]:

import tensorflow as tf
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

2.0.0
Found GPU at: /device:GPU:0


In [2]:
# import the necessary packages
import cv2
import os
import time
import datetime
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # additional plotting functionality
from imutils import paths
from glob import glob

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3, Xception, MobileNet, NASNetLarge, ResNet50
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Load the TensorBoard notebook extension
%load_ext tensorboard

#import argparse
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

## Set path variables

In [3]:
import os
os.chdir("D:/datasets/NIH/")
DATA_DIR = os.getcwd()
#DATA_DIR 'D:\\datasets\\NIH'

## Labels

In [ ]:
labels_df = pd.read_csv('Data_Entry_2017.csv', low_memory=False)

# see how many observations there are
num_obs = len(labels_df)
print('Number of observations:',num_obs)

# examine the raw data before performing pre-processing
labels_df.head(5) # view first 5 rows

In [ ]:
# Find unique labels
num_unique_labels = labels_df['Finding Labels'].nunique()
print('Number of unique labels:',num_unique_labels)

In [ ]:
# Label distributions
count_per_unique_label = labels_df['Finding Labels'].value_counts() # get frequency counts per label
df_count_per_unique_label = count_per_unique_label.to_frame() # convert series to dataframe for plotting purposes

print(df_count_per_unique_label) # view tabular results

In [ ]:
sns.barplot(x = df_count_per_unique_label.index[:20], 
            y="Finding Labels", 
            data=df_count_per_unique_label[:20], 
            color = "green"), 
plt.xticks(rotation = 90) # visualize results graphically

In [ ]:
# define dummy labels for one hot encoding - simplifying to 14 primary classes (excl. No Finding)
dummy_labels = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 
'Cardiomegaly', 'Nodule', 'Mass', 'Hernia'] # taken from paper

# One Hot Encoding of Finding Labels to dummy_labels
for label in dummy_labels:
    labels_df[label] = labels_df['Finding Labels'].map(lambda result: 1.0 if label in result else 0)
labels_df.head() # check the data, looking good!

## Images

In [ ]:
data_image_paths = {os.path.basename(x): x for x in glob(os.path.join(DATA_DIR, 'images*', '*', '*.png'))}
print('Number of Observations: ', len(data_image_paths)) # check to make sure I've captured every pathway, should equal 112,120 

In [ ]:
labels_df['Finding Labels'] = labels_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))

In [ ]:
from itertools import chain

labels = np.unique(list(chain(*labels_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
labels = [x for x in labels if len(x) > 0]

print(len(labels))
labels

In [ ]:
# One hot encode labels
for label in labels:
    if len(label) > 1:
        labels_df[label] = labels_df['Finding Labels'].map(lambda finding: 1.0 if label in finding else 0.0)

In [ ]:
# Drop label 'Hernia' as it is not a disease

labels = [label for label in labels if labels_df[label].sum() > 1000]

print(len(labels))
labels

In [ ]:
train_df, valid_df = train_test_split(labels_df, test_size=0.20, 
                                      random_state=89, 
                                      stratify=labels_df['Finding Labels'].map(lambda x: x[:4]))

In [ ]:
train_df['labels'] = train_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)
valid_df['labels'] = valid_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)

In [ ]:
core_idg = ImageDataGenerator(rescale=1 / 255,
                                  samplewise_center=True,
                                  samplewise_std_normalization=True,
                                  horizontal_flip=True,
                                  vertical_flip=False,
                                  height_shift_range=0.05,
                                  width_shift_range=0.1,
                                  rotation_range=5,
                                  shear_range=0.1,
                                  fill_mode='reflect',
                                  zoom_range=0.15)

train_gen = core_idg.flow_from_dataframe(dataframe=train_df,
                                             directory=None,
                                             x_col='path',
                                             y_col='labels',
                                             class_mode='categorical',
                                             batch_size=BATCH_SIZE,
                                             classes=labels,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE))

valid_gen = core_idg.flow_from_dataframe(dataframe=valid_df,
                                             directory=None,
                                             x_col='path',
                                             y_col='labels',
                                             class_mode='categorical',
                                             batch_size=BATCH_SIZE,
                                             classes=labels,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE))

test_X, test_Y = next(core_idg.flow_from_dataframe(dataframe=valid_df,
                                                       directory=None,
                                                       x_col='path',
                                                       y_col='labels',
                                                       class_mode='categorical',
                                                       batch_size=1024,
                                                       classes=labels,
                                                       target_size=(IMAGE_SIZE, IMAGE_SIZE)))

# pretrained inceptionnetv3
https://www.kaggle.com/levanpon1009/nih-chest-x-rays-with-inceptionresnetv2

In [4]:

image_size = 256
batch_size = 32

In [5]:
# load data
all_xray_df = pd.read_csv('Data_Entry_2017.csv')

In [6]:


data_image_paths = {os.path.basename(x): x for x in glob(os.path.join(DATA_DIR, 'images*', '*', '*.png'))}



In [7]:
all_xray_df['path'] = all_xray_df['Image Index'].map(data_image_paths.get)

In [8]:
all_xray_df['path']

0         D:\datasets\NIH\images_001\images\00000001_000...
1         D:\datasets\NIH\images_001\images\00000001_001...
2         D:\datasets\NIH\images_001\images\00000001_002...
3         D:\datasets\NIH\images_001\images\00000002_000...
4         D:\datasets\NIH\images_001\images\00000003_000...
                                ...                        
112115    D:\datasets\NIH\images_012\images\00030801_001...
112116    D:\datasets\NIH\images_012\images\00030802_000...
112117    D:\datasets\NIH\images_012\images\00030803_000...
112118    D:\datasets\NIH\images_012\images\00030804_000...
112119    D:\datasets\NIH\images_012\images\00030805_000...
Name: path, Length: 112120, dtype: object

In [9]:
all_xray_df['Finding Labels'] = all_xray_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))

In [10]:
from itertools import chain

labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
labels = [x for x in labels if len(x) > 0]

#labels

In [11]:
for label in labels:
    if len(label) > 1:
        all_xray_df[label] = all_xray_df['Finding Labels'].map(lambda finding: 1.0 if label in finding else 0.0)

In [12]:
labels = [label for label in labels if all_xray_df[label].sum() > 1000]

In [13]:
train_df, valid_df = train_test_split(all_xray_df, test_size=0.20, 
                                      random_state=89, 
                                      stratify=all_xray_df['Finding Labels'].map(lambda x: x[:4]))

In [14]:

train_df['labels'] = train_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)
valid_df['labels'] = valid_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)

C:\Users\mthompson1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\mthompson1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
'''
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

'''
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(len(labels), activation="sigmoid")(x)
model = tf.keras.Model(base_model.input, output)
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
core_idg = ImageDataGenerator(rescale=1 / 255,
                                  samplewise_center=True,
                                  samplewise_std_normalization=True,
                                  horizontal_flip=True,
                                  vertical_flip=False,
                                  height_shift_range=0.05,
                                  width_shift_range=0.1,
                                  rotation_range=5,
                                  shear_range=0.1,
                                  fill_mode='reflect',
                                  zoom_range=0.15)

train_gen = core_idg.flow_from_dataframe(dataframe=train_df,
                                             directory=None,
                                             x_col='path',
                                             y_col='labels',
                                             class_mode='categorical',
                                             batch_size=batch_size,
                                             classes=labels,
                                             target_size=(image_size, image_size))

valid_gen = core_idg.flow_from_dataframe(dataframe=valid_df,
                                             directory=None,
                                             x_col='path',
                                             y_col='labels',
                                             class_mode='categorical',
                                             batch_size=batch_size,
                                             classes=labels,
                                             target_size=(image_size, image_size))

test_X, test_Y = next(core_idg.flow_from_dataframe(dataframe=valid_df,
                                                       directory=None,
                                                       x_col='path',
                                                       y_col='labels',
                                                       class_mode='categorical',
                                                       batch_size=1024,
                                                       classes=labels,
                                                       target_size=(image_size, image_size)))

Found 41318 validated image filenames belonging to 13 classes.
Found 10331 validated image filenames belonging to 13 classes.
Found 10331 validated image filenames belonging to 13 classes.


In [ ]:
'''
def get_callbacks(model_name):
    callbacks = []
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    callbacks.append(tensorboard_callback)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'model.{model_name}.h5',
        verbose=1,
        save_best_only=True)
    # erly = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    callbacks.append(checkpoint)
    # callbacks.append(erly)
    return callbacks
    
    
callbacks = get_callbacks('inceptionresnetv2')


model.fit(train_gen,
              steps_per_epoch=100,
              validation_data=(test_X, test_Y),
              epochs=50,
              callbacks=callbacks)
'''   

In [17]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [21]:
startTrainTime = time.time()

# train the model
# this model uses balanced batch generator

model_train = model.fit_generator(train_gen,
                                  
                                  epochs = 15,
                                  validation_data = (test_X, test_Y))

endTrainTime = time.time()
trainTime = endTrainTime - startTrainTime
print()
print('Total Training Time (sec): {}'.format(trainTime))

Epoch 1/15


ResourceExhaustedError: OOM when allocating tensor with shape[32,64,29,29] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

# Jeff's for loop

In [ ]:
def model_picker(name):
#
#  """Function allows the choice of any pretrained model contained within the 
#  Keras architecture but without the top classification layers so only the 
#  bottleneck features are retained.
#
#  Args: 
#      Model name (str): The name of the desired model
#
#  Returns: 
#      Selected model
#  """
#
  if (name == 'VGG16'):
        model = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
        
  elif (name == 'VGG19'):
        model = VGG19(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
        
  elif (name == 'MobileNet'):
        model = MobileNet(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
        
  elif (name == 'InceptionV3'):
        model = InceptionV3(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
        
  elif (name == 'ResNet50'):
        model = ResNet50(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
        
  elif (name == 'Xception'):
        model = Xception(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
        
  else:
        print("Specified model not available")
  return model

In [ ]:
# train the head of the network
model = ['VGG16','VGG19','MobileNet','InceptionV3','ResNet50','Xception']

# initialize the training data augmentation object
#trainAug = ImageDataGenerator(rotation_range=15,fill_mode="nearest")

for model_str in model:
    
#    model = model_picker(model)
    
    # place the head FC model on top of the base model (this will become
    # the actual model we will train)
    
    baseModel = model_picker(model_str)
    
    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(64, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(13, activation="softmax")(headModel)
    
    model = Model(inputs=baseModel.input, outputs=headModel)
    
    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    for layer in baseModel.layers:
        layer.trainable = False
        
    # compile our model
    INIT_LR = 1e-3
    EPOCHS = 25
    BS = 8
    
    print("[INFO] compiling model..."+model_str)
    opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    
    

    callbacks = get_callbacks(model)
    
    model.fit(train_gen,
              steps_per_epoch=100,
              validation_data=(test_X, test_Y),
              epochs=50,
              callbacks=callbacks)


    y_pred = model.predict(test_X)
    
    for label, p_count, t_count in zip(labels,
                                       100 * np.mean(y_pred, 0),
                                       100 * np.mean(test_Y, 0)):
        print('%s: actual: %2.2f%%, predicted: %2.2f%%' % (label, t_count, p_count))    print("[INFO] training head...")
    H = model.fit_generator(
        train_gen.flow(train_gen, valid_gen, batch_size=BS),
        steps_per_epoch=len(train_gen) // BS,
        validation_data=(testX, testY),
        validation_steps=len(testX) // BS,
        epochs=EPOCHS)
    
    # make predictions on the testing set
    print("[INFO] evaluating network...")
    predIdxs = model.predict(testX, batch_size=BS)
    
    
    # for each image in the testing set we need to find the index of the
    # label with corresponding largest predicted probability
    predIdxs = np.argmax(predIdxs, axis=1)
    
    # show a nicely formatted classification report
    print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))
    
    
    # compute the confusion matrix and and use it to derive the raw
    # accuracy, sensitivity, and specificity
    
    cm = confusion_matrix(testY.argmax(axis=1), predIdxs)
    total = sum(sum(cm))
    acc = (cm[0, 0] + cm[1, 1]) / total
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    
    
    # show the confusion matrix, accuracy, sensitivity, and specificity
    print(cm)
    print("acc: {:.4f}".format(acc))
    print("sensitivity: {:.4f}".format(sensitivity))
    print("specificity: {:.4f}".format(specificity))
    
    # plot the training loss and accuracy
    N = EPOCHS
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy on NIH Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig('plot')
    plt.show()
    
    # serialize the model to disk
    print("[INFO] saving NIH detector model...")
    model.save('NIH_model_'+model_str)

# Resampled & Normalized 
https://www.kaggle.com/kmader/create-a-mini-xray-dataset-equalized

In [ ]:
all_xray_df = pd.read_csv('Data_Entry_2017.csv')
all_image_paths = {os.path.basename(x): x for x in glob(os.path.join('images*/images/*.png'))}
print('Scans found:', len(all_image_paths), ', Total Headers', all_xray_df.shape[0])

all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
all_xray_df['Cardiomegaly'] = all_xray_df['Finding Labels'].map(lambda x: 'Cardiomegaly' in x)
all_xray_df['Patient Age'] = np.clip(all_xray_df['Patient Age'], 5, 100)
all_xray_df['Patient Male'] = all_xray_df['Patient Gender'].map(lambda x: x.upper()=='M').astype('float32')
all_xray_df.sample(3)


In [ ]:
label_counts = all_xray_df['Finding Labels'].value_counts()[:15]
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [ ]:
all_xray_df['Finding Labels'] = all_xray_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
from itertools import chain
all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
print('All Labels', all_labels)
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        all_xray_df[c_label] = all_xray_df['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)
all_xray_df.sample(3)

In [ ]:
# since we can't have everything make a nice subset
# weight is 0.1 + number of findings
sample_weights = all_xray_df['Finding Labels'].map(lambda x: len(x.split('|')) if len(x)>0 else 0).values + 1e-1
sample_weights /= sample_weights.sum()
all_xray_df = all_xray_df.sample(18000, weights=sample_weights)

label_counts = all_xray_df['Finding Labels'].value_counts()[:15]
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [ ]:
import h5py
from tqdm import tqdm

def write_df_as_hdf(out_path,
                    out_df,
                    compression='gzip'):
    with h5py.File(out_path, 'w') as h:
        for k, arr_dict in tqdm(out_df.to_dict().items()):
            try:
                s_data = np.stack(arr_dict.values(), 0)
                try:
                    h.create_dataset(k, data=s_data, compression=
                    compression)
                except TypeError as e:
                    try:
                        h.create_dataset(k, data=s_data.astype(np.string_),
                                         compression=compression)
                    except TypeError as e2:
                        print('%s could not be added to hdf5, %s' % (
                            k, repr(e), repr(e2)))
            except ValueError as e:
                print('%s could not be created, %s' % (k, repr(e)))
                all_shape = [np.shape(x) for x in arr_dict.values()]
                warn('Input shapes: {}'.format(all_shape))

In [ ]:
write_df_as_hdf('chest_xray.h5', all_xray_df)

In [ ]:


# show what is inside
with h5py.File('chest_xray.h5', 'r') as h5_data:
    for c_key in h5_data.keys():
        print(c_key, h5_data[c_key].shape, h5_data[c_key].dtype)



In [ ]:


from skimage.transform import resize
OUT_DIM = (128, 128)
def imread_and_normalize(im_path):
    clahe_tool = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_data = np.mean(cv2.imread(im_path), 2).astype(np.uint8)
    img_data = clahe_tool.apply(img_data)
    n_img = (255*resize(img_data, OUT_DIM, mode = 'constant')).clip(0,255).astype(np.uint8)
    return np.expand_dims(n_img, -1)

test_img = imread_and_normalize(all_xray_df['path'].values[0])
plt.matshow(test_img[:,:,0])



In [ ]:
%%time
# time for 100 images
pre_read_tiles = np.stack(all_xray_df.sample(100)['path'].map(imread_and_normalize).values,0)

In [ ]:
# might as well show-em if we have em
from skimage.util import montage
fig, ax1 = plt.subplots(1,1, figsize = (12,12))
ax1.imshow(montage(pre_read_tiles[:,:,:,0]), cmap = 'bone')
fig.savefig('overview.png', dpi = 300)

In [ ]:
# preallocate output
out_image_arr = np.zeros((all_xray_df.shape[0],)+OUT_DIM+(1,), dtype=np.uint8)
if False:
    # a difficult to compress array for size approximations
    out_image_arr = np.random.uniform(0, 255,
                                  size = (all_xray_df.shape[0],)+OUT_DIM+(1,)).astype(np.uint8)

In [ ]:
for i, c_path in enumerate(tqdm(all_xray_df['path'].values)):
    out_image_arr[i] = imread_and_normalize(c_path)

In [ ]:
# append the array
with h5py.File('chest_xray.h5', 'a') as h5_data:
    h5_data.create_dataset('images', data = out_image_arr, compression = None) # compression takes too long
    for c_key in h5_data.keys():
        print(c_key, h5_data[c_key].shape, h5_data[c_key].dtype)